In [7]:
from pathlib import Path

import polars as pl
from sqlmodel import Session, create_engine, select

# DESDEO models (adjust import paths if your package layout differs)
from desdeo.api.models.generic_states import State, StateDB, StateKind
from desdeo.api.models.state import ENautilusState  # concrete table for ENautilus

DB_PATH = Path("./test.db")
engine = create_engine(f"sqlite:///{DB_PATH}", echo=False)

db = Session(engine)

In [9]:
PROBLEM_ID = 4

stmt = (
    select(StateDB, State, ENautilusState)
    .join(State, StateDB.state_id == State.id)
    .join(ENautilusState, ENautilusState.id == State.id)
    .where(StateDB.problem_id == PROBLEM_ID)
    .where(State.kind == StateKind.ENAUTILUS_STEP)
)

rows = db.exec(stmt).all()
rows

[(StateDB(problem_id=4, session_id=None, state_id=1, id=1, parent_id=None), State(kind=<StateKind.ENAUTILUS_STEP: 'e-nautilus.stepping'>, method='e-nautilus', phase='stepping', id=1), ENautilusState(id=1, iterations_left=5, reachable_point_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, ... (42679 characters truncated) ...  1999]]), non_dominated_solutions_id=1, current_iteration=0, selected_point={'f_1': 0.0, 'f_2': 6.0, 'f_3': 1503.29}, number_of_intermediate_points=3)),
 (StateDB(problem_id=4, session_id=None, state_id=2, id=2, parent_id=1), State(kind=<StateKind.ENAUTILUS_STEP: 'e-nautilus.stepping'>, method='e-nautilus', phase='stepping', id=2), ENautilusState(id=2, iterations_left=4, reachable_point_indices=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, ... (39034 characters truncated) ... ons_id=1, current_iteration=1, selected_point={'f_1': 28.400000000000002, 'f_2': 4.800000000000001, 'f_3':

In [ ]:
def rows_to_nodes_df(rows) -> pl.DataFrame:
    recs = []
    for sdb, st, ena in rows:
        recs.append(
            {
                "statedb_id": sdb.id,
                "parent_statedb_id": sdb.parent_id,
                "session_id": sdb.session_id,
                "problem_id": sdb.problem_id,
                "state_id": st.id,
                "method": st.method,
                "phase": st.phase,
                "kind": st.kind,
                "current_iteration": ena.current_iteration,
                "iterations_left": ena.iterations_left,
                "number_of_intermediate_points": ena.number_of_intermediate_points,
                # DM-side choice at this node (child node selected_point is the choice made)
                "selected_point": ena.selected_point,
                # Sometimes useful as quick complexity/option set proxy
                "reachable_point_indices": ena.reachable_point_indices,
                # Keep the decoded object; we will mine it for intermediate points etc.
                "enautilus_results": ena.enautilus_results,
            }
        )
    return pl.DataFrame(recs)


nodes = rows_to_nodes_df(rows)
nodes.sort(["session_id", "statedb_id"]).head(5)

shape: (3, 14)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ statedb_i ┆ parent_st ┆ session_i ┆ problem_i ┆ … ┆ number_of ┆ selected_ ┆ reachable ┆ enautilu │
│ d         ┆ atedb_id  ┆ d         ┆ d         ┆   ┆ _intermed ┆ point     ┆ _point_in ┆ s_result │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ iate_poin ┆ ---       ┆ dices     ┆ s        │
│ i64       ┆ i64       ┆ null      ┆ i64       ┆   ┆ ts        ┆ struct[3] ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆           ┆ list[i64] ┆ object   │
│           ┆           ┆           ┆           ┆   ┆ i64       ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1         ┆ null      ┆ null      ┆ 4         ┆ … ┆ 3         ┆ {0.0,6.0, ┆ [0, 1, …  ┆ current_ │
│           ┆           ┆           ┆           ┆   ┆           ┆ 1503.29}  ┆ 1999]     ┆ iteratio │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ n=1 iter │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ations…  │
│ 2         ┆ 1         ┆ null      ┆ 4         ┆ … ┆ 3         ┆ {28.4,4.8 ┆ [0, 1, …  ┆ current_ │
│           ┆           ┆           ┆           ┆   ┆           ┆ ,1238.288 ┆ 1999]     ┆ iteratio │
│           ┆           ┆           ┆           ┆   ┆           ┆ }         ┆           ┆ n=2 iter │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ations…  │
│ 3         ┆ 2         ┆ null      ┆ 4         ┆ … ┆ 3         ┆ {135.55,3 ┆ [0, 1, …  ┆ current_ │
│           ┆           ┆           ┆           ┆   ┆           ┆ .6,1120.3 ┆ 1999]     ┆ iteratio │
│           ┆           ┆           ┆           ┆   ┆           ┆ 6725}     ┆           ┆ n=3 iter │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆ ations…  │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [ ]:
def build_children_map(nodes: pl.DataFrame) -> dict[int, list[int]]:
    children = {}
    for sid, pid in nodes.select(["statedb_id", "parent_statedb_id"]).iter_rows():
        children.setdefault(sid, [])
        if pid is not None:
            children.setdefault(pid, []).append(sid)
    for k in children:
        children[k].sort()
    return children


def find_roots(nodes: pl.DataFrame) -> list[int]:
    sids = set(nodes["statedb_id"].to_list())
    pids = set([p for p in nodes["parent_statedb_id"].to_list() if p is not None])
    # roots are those whose parent is NULL or not present in this filtered set
    roots = [
        sid
        for sid in sids
        if sid not in pids and nodes.filter(pl.col("statedb_id") == sid)["parent_statedb_id"][0] is None
    ]
    roots.sort()
    return roots


children = build_children_map(nodes)
roots = find_roots(nodes)


def all_paths(children: dict[int, list[int]], roots: list[int]) -> list[list[int]]:
    paths = []

    def dfs(node: int, acc: list[int]):
        kids = children.get(node, [])
        if not kids:
            paths.append(acc + [node])
            return
        for c in kids:
            dfs(c, acc + [node])

    for r in roots:
        dfs(r, [])
    return paths


paths = all_paths(children, roots)
len(paths), paths[:2]

paths

[]

In [ ]:
import math


def point_keys_from_results(res) -> list[str]:
    # intermediate_points are dicts keyed by objective symbols
    if res is None or not getattr(res, "intermediate_points", None):
        return []
    return sorted(res.intermediate_points[0].keys())


def vec_from_point(p: dict, keys: list[str]) -> list[float]:
    return [float(p[k]) for k in keys]


def match_choice(parent_res, child_selected, tol=1e-6):
    if parent_res is None or child_selected is None:
        return None, None  # (index, distance)

    inter = getattr(parent_res, "intermediate_points", None)
    if not inter:
        return None, None

    keys = sorted(inter[0].keys())
    target = vec_from_point(child_selected, keys)

    best_i, best_d = None, float("inf")
    for i, p in enumerate(inter):
        v = vec_from_point(p, keys)
        d = math.sqrt(sum((a - b) ** 2 for a, b in zip(v, target)))
        if d < best_d:
            best_d, best_i = d, i

    if best_d <= tol:
        return best_i, best_d
    return None, best_d

In [15]:
# index nodes by statedb_id for fast access
node_map = {r["statedb_id"]: r for r in nodes.to_dicts()}

choice_recs = []
for parent_id, kids in children.items():
    parent = node_map.get(parent_id)
    if parent is None:
        continue

    parent_res = parent["enautilus_results"]

    for child_id in kids:
        child = node_map.get(child_id)
        if child is None:
            continue

        chosen_idx, dist = match_choice(parent_res, child["selected_point"], tol=1e-6)

        rec = {
            "session_id": parent["session_id"],
            "problem_id": parent["problem_id"],
            "parent_statedb_id": parent_id,
            "child_statedb_id": child_id,
            "parent_iteration": parent["current_iteration"],
            "child_iteration": child["current_iteration"],
            "chosen_intermediate_index": chosen_idx,
            "match_distance": dist,
        }

        if chosen_idx is not None:
            rec |= {
                "chosen_point": parent_res.intermediate_points[chosen_idx],
                "chosen_closeness": parent_res.closeness_measures[chosen_idx],
                "chosen_reachable_set_size": len(parent_res.reachable_point_indices[chosen_idx]),
                "chosen_best_bounds": parent_res.reachable_best_bounds[chosen_idx],
                "chosen_worst_bounds": parent_res.reachable_worst_bounds[chosen_idx],
            }

        choice_recs.append(rec)

choices = pl.DataFrame(choice_recs)
choices.sort(["session_id", "parent_iteration", "parent_statedb_id"]).head(10)

session_id,problem_id,parent_statedb_id,child_statedb_id,parent_iteration,child_iteration,chosen_intermediate_index,match_distance,chosen_point,chosen_closeness,chosen_reachable_set_size,chosen_best_bounds,chosen_worst_bounds
null,i64,i64,i64,i64,i64,i64,f64,struct[3],f64,i64,struct[3],struct[3]
null,4,1,2,0,1,1,0.0,"{28.4,4.8,1238.288}",20.0,1917,"{524.0,0.0,89.14}","{28.4,4.8,1238.288}"
null,4,2,3,1,2,2,0.0,"{135.55,3.6,1120.36725}",46.855859,1723,"{508.0,0.0,178.28}","{135.55,3.6,1120.36725}"


In [17]:
choices.group_by("session_id").agg(
    pl.len().alias("num_steps"),
    pl.col("chosen_closeness").drop_nulls().mean().alias("mean_closeness"),
    pl.col("chosen_reachable_set_size").drop_nulls().mean().alias("mean_reachable_set_size"),
)

session_id,num_steps,mean_closeness,mean_reachable_set_size
null,u32,f64,f64
null,2,33.427929,1820.0
